![](https://i.imgur.com/eBRPvWB.png)

# Practical PyTorch: Summarization with a Sequence to Sequence Network and Attention on Language Model

This is made possible by the simple but powerful idea of the [sequence to sequence network](http://arxiv.org/abs/1409.3215), in which two recurrent neural networks work together to transform one sequence to another. An encoder network condenses an input sequence into a single vector, and a decoder network unfolds that vector into a new sequence. 

To improve upon this model we'll use an [attention mechanism](https://arxiv.org/abs/1409.0473), which lets the decoder learn to focus over a specific range of the input sequence. This is initially designed for data-driven language translation, in which case the length of input and output sequence is approximately similar. For text summarization task, in which case the output sequence is much shorter then the input, the model didn't perform that well. Some other tricks should be explored soon. 

# The Sequence to Sequence model

A [Sequence to Sequence network](http://arxiv.org/abs/1409.3215), or seq2seq network, or [Encoder Decoder network](https://arxiv.org/pdf/1406.1078v3.pdf), is a model consisting of two separate RNNs called the **encoder** and **decoder**. The encoder reads an input sequence one item at a time, and outputs a vector at each step. The final output of the encoder is kept as the **context** vector. The decoder uses this context vector to produce a sequence of outputs one step at a time.

## The Attention Mechanism

The fixed-length vector carries the burden of encoding the the entire "meaning" of the input sequence, no matter how long that may be. With all the variance in language, this is a very hard problem. Imagine two nearly identical sentences, twenty words long, with only one word different. Both the encoders and decoders must be nuanced enough to represent that change as a very slightly different point in space.

The **attention mechanism** [introduced by Bahdanau et al.](https://arxiv.org/abs/1409.0473) addresses this by giving the decoder a way to "pay attention" to parts of the input, rather than relying on a single vector. For every step the decoder can select a different part of the input sentence to consider.

# Requirements

You will need [PyTorch](http://pytorch.org/) to build and train the models, and [matplotlib](https://matplotlib.org/) for plotting training and visualizing attention outputs later.

In [ ]:
import unicodedata
import string
import re
import random
import time
import math
import datetime
import csv

'''
import socket

hostname = socket.gethostname()
'''

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence#, masked_cross_entropy
from masked_cross_entropy import *
torch.version.cuda

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
%matplotlib inline


import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [ ]:
USE_CUDA = False

## Loading Data

Similar to the character encoding used in the character-level RNN tutorials, we will be representing each word in a language as a one-hot vector, or giant vector of zeros except for a single one (at the index of the word). Compared to the dozens of characters that might exist in a language, there are many many more words, so the encoding vector is much larger. We will however cheat a bit and trim the data to only use a few thousand words per language. On other tutorials,  they adopt an initial embedding matrix built from [GloVe](http://nlp.stanford.edu/projects/glove/), which can be tested afterwards.

### Sentence Embeddings
We'll use the gensim's library of pretrained sentence-to-vector models originally trained on the AP News dataset to use as a sentence-level analogue to semantic word embeddings. This is used as the second layer of the hierarchical attention model.

In [ ]:
import gensim
sentence_size = 100

sent2vec = gensim.models.doc2vec.Doc2Vec.load("/Data/apnews_model/apnews_sen_model.model")

In [ ]:
embedding = sent2vec.infer_vector('Hello my name is friend.')
print(len(embedding), embedding)

In [ ]:
from torchtext import data
import torchtext.vocab as vocab
from torchtext.data import RawField

### Only for data visualization, does not need to be done

In [ ]:
import pandas as pd
# note. this is the wrong dataset right now. (I think)
data_df = pd.read_csv('/Data/utiman/wiki_queries12.csv')
data_df.head()

In [ ]:
tensor = torch.FloatTensor(embedding)
if USE_CUDA:
    tensor = tensor.cuda()
tensor[:5]

In [ ]:
from scipy import sparse

vectors = sparse.load_npz('/Data/utiman/data/queries_matrix.npz')

In [ ]:
def postprocess(batch):#, device, train):
    """ Process a list of examples to create a torch.Tensor.
    Pad, numericalize, and postprocess a batch and create a tensor.
    Args:
        batch (list(object)): A list of object from a batch of examples.
    Returns:
        torch.autograd.Variable: Processed object given the input
            and custom postprocessing Pipeline.
    """
    # vectors_file = sparse.load_npz('../data/queries_matrix.npz')

    csr_vec = vectors[batch[0]]
    vec = sparse.csr_matrix.todense(csr_vec)
    tensor = torch.LongTensor(vec)
    
    ret_matrix = tensor
    
    for q_idx in batch[1::]:
        #print(q_idx)
        csr_vec = vectors[batch[0]]
        vec = sparse.csr_matrix.todense(csr_vec)
        tensor = torch.LongTensor(vec)
        ret_matrix = torch.cat((ret_matrix, tensor),1)
    
    #if device != -1:
    if USE_CUDA:
        ret_matrix = ret_matrix.contiguous().cuda()
    
    return torch.autograd.Variable(ret_matrix)

In [ ]:
import nltk

In [ ]:
'''
    @args:
        batch: a list of strings where each string is a document to be converted to sen_vec 
            representation
    @returns: np.array that is (batch_size x num_sens_in_doc x 100) 
                This array si the sentences embeddings for each sentence 
                in each document in the batch
'''
def sen_vec_postprocess(batch):
    
    # tokenized batch is now a list[list[sentences]]
    # split at <sos> tokens
    tokenized_batch = [['<sos>' + sen for sen in example.split('<sos>')][1:] for example in batch]
    print('my tokenize: ', tokenized_batch)

    # maximum length of any document (in number of sentences)
    max_doc_len = np.max(np.array([len(ex) for ex in batch]))
    
    batch_vec = np.zeros((len(batch), max_doc_len, 100))
    
    for i, example in enumerate(tokenized_batch):
        # length of this example tells us how much we need to leave as padding on the end
        for j in range(len(example)):
            batch_vec[i,:] = np.array(sent2vec.infer_vector(example[j]))

    # return as cuda var
    tensor = torch.FloatTensor(batch_vec)
    print(tensor[:,:,10])
    if USE_CUDA:
        tensor = tensor.cuda()
    return torch.autograd.Variable(tensor)

In [ ]:
'''
    @args:
        batch: a list of strings where each string is a document to be converted to sen_vec 
            representation
    @returns: np.array that is (batch_size x num_sens_in_doc x 100) 
                This array si the sentences embeddings for each sentence 
                in each document in the batch
'''
def sen_idx_postprocess(batch):
    # tokenized batch is now a list[list[sentences]]
    # split at <sos> tokens
    sentences_batch = [['<sos>' + sen for sen in example.split('<sos>')][1:] for example in batch]

    # words batch is now a list[list[words]]
    words_batch = [example.split() for example in batch]

    # maximum length of any document (in number of words)
    max_doc_len = max([len(ex) for ex in words_batch])
    
    # maximum number of sentences in any document
    max_sen_num = max([len(ex) for ex in sentences_batch])

    # initialize the array of sentence indices
    # we use one more than the maximum to be the zero vector.
    # In sentence attentions, this always receives zero weight
    sen_idxs = np.ones((len(batch), max_doc_len)) * max_sen_num
    
    # for ea
    for sen_idx, example in enumerate(sentences_batch):
        curr = 0
        for wd_idx, sent in enumerate(example):
            sen_idxs[sen_idx, curr:curr+len(sent.split())] = wd_idx
            curr += len(sent.split())

    if USE_CUDA:
        return torch.autograd.Variable(torch.LongTensor(sen_idxs).cuda())
    else:
        return torch.autograd.Variable(torch.LongTensor(sen_idxs))

In [ ]:
batch = ["<sos> lorem ipsum dolor sit amet <sos> consectetur adipiscing elit <sos> sed do eiusmod tempor incididunt <sos> ut labore et dolore magna aliqua",
         "<sos> ut enim ad minim veniam , quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat <sos> Duis aute irure dolor in reprehenderit in voluptate velit <sos> esse cillum dolore eu fugiat nulla pariatur"]

sen_idx_postprocess(batch)

In [ ]:
batch = ["<sos> hello my name is friend"]
v = sen_vec_postprocess(batch).squeeze()
u = sen_vec_postprocess(batch).squeeze()
v = v.data.cpu().numpy()
u = u.data.cpu().numpy()
from scipy import spatial
#print(dist = 1.0 - np.dot(u, v) / (norm(u) * norm(v)))
print(spatial.distance.cosine(u,v))

In [ ]:
TEXT = data.Field(sequential=True, lower=True)

In [ ]:
TEXT

In [ ]:
QUERY = data.RawField(postprocessing=postprocess) # sequential=False, use_vocab = False, 

In [ ]:
SEN_VEC = data.RawField(postprocessing=sen_vec_postprocess) #, sequential=False, use_vocab = False)

In [ ]:
SEN_IDX = data.RawField(postprocessing=sen_idx_postprocess)

In [ ]:
import csv
# need to do this so we dont get errors about having too big of a file in a single cell of a csv
csv.field_size_limit(500 * 1024 * 1024)

In [ ]:
train, = data.TabularDataset.splits(
        path='/Data/utiman/', train='wiki_queries12.csv',format='csv', skip_header = True,
        fields=[('query_num', QUERY), ('title', TEXT), ('raw_query', TEXT), ('sum', TEXT),
                ('story', TEXT), ('sen_vec', SEN_VEC), ('sen_idx', SEN_IDX)])

##### Small dataset (testing)

In [ ]:
train, = data.TabularDataset.splits(
        path='/home/wdv2/ats/data/', train='wiki_queries12_head.csv',format='csv', skip_header = True,
        fields=[('query_num', QUERY), ('title', TEXT), ('raw_query', TEXT), ('sum', TEXT),
                ('story', TEXT), ('sen_vec', SEN_VEC), ('sen_idx', SEN_IDX)])

In [ ]:
example_q = train.examples[1].raw_query

example_sum = train.examples[1].sum
example_doc = train.examples[1].story
ex = train.examples[1].sen_idx
#print(ex)
#' '.join(example_doc)
#print(len(example_q), len(example_sum), len(example_doc))
#ex

print ('QUERY_NUM:  ', train.examples[1].query_num)
print ('TITLE:      ', train.examples[1].title)
print ('RAW_QUERY:  ', train.examples[1].raw_query)
print ('SUMMARY:    ', train.examples[1].sum)
print ('STORY_LEN:  ', len(train.examples[1].story))
print ('STORY_HEAD: ', train.examples[1].story[:100])

### Extract Vocabulatry
We get the vocabulary of the data using 100-dimensional GloVe vectors with the torchtext module.

In [ ]:
TEXT.build_vocab(train, vectors="glove.6B.100d")

To read the data file we will split the file into lines, and then split lines into pairs. The files are all description &rarr; headline, so if we want to generate text from headline &rarr; description I added the `reverse` flag to reverse the pairs.

In [ ]:
vocab = TEXT.vocab
vocab.vectors.shape
SOS_token = vocab.stoi['<SOS>']
EOS_token = vocab.stoi['<EOS>']
#vocab.stoi
print('SOS token:{}, EOS token:{}'.format(SOS_token, EOS_token))

The full process for preparing the data is:

* Read text file and split into lines, split lines into pairs
* Normalize text, filter by length and content
* Make word lists from sentences in pairs

In [ ]:
def get_iterator_over_batches(dataset, batch_size, train=True, shuffle=True, repeat=False):
    dataset_iter = data.BucketIterator(
        dataset, batch_size=batch_size, device=-1, 
        sort_key=lambda x: len(x.story),
        train=train, shuffle=shuffle, repeat=repeat,
        sort=False
    )
    dataset_iter.create_batches()
    return dataset_iter

#make bucket iterator 
b_iter = get_iterator_over_batches(train, 32)
for batch in b_iter:
    print('size of batch' , batch.batch_size, '|| len of examples in batch', len(batch.story))
    
    # examples come out of the batch numericalized in a batch of (num_words_in_padded_sen X batch_size)
#     print("Numericalized Batch")
#     print(batch.story)
    
    # We can decode an example back into a list of words to see that these values are reasonable
    #example = batch.story[:,0].data.tolist()
    #print('decoded example from batch:')
    #print([vocab.itos[e] for e in example])
    
    # We can create a example embedding layer to see what happens when we use the glove vectors properly
    embed = nn.Embedding(len(vocab), 100)
    embed.weight.data.copy_(vocab.vectors)
    
    print("Shape of embedded batch")
    embedding = embed(batch.story) # Embedded batch will be (num_words X batch_size X glove_dim)   
    print(embedding.shape)
    
    print("Sentence vectors batch shape:{}".format(batch.sen_vec.shape))
    print(batch.sen_vec) # batch_size x 
    
    print ("Sentence idx")
    print (batch.sen_idx)
    
    break

# Building the models

## The Encoder

<img src="images/encoder-network.png" style="float: right" />

The encoder will take a batch of word sequences, a `LongTensor` of size `(max_len x batch_size)`, and output an encoding for each word, a `FloatTensor` of size `(max_len x batch_size x hidden_size)`.

The word inputs are fed through an [embedding layer `nn.Embedding`](http://pytorch.org/docs/nn.html#embedding) to create an embedding for each word, with size `seq_len x hidden_size` (as if it was a batch of words). This is resized to `seq_len x 1 x hidden_size` to fit the expected input of the [GRU layer `nn.GRU`](http://pytorch.org/docs/nn.html#gru). The GRU will return both an output sequence of size `seq_len x hidden_size`.

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1, dropout=0.1):
        super(EncoderRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.embedding.weight.data.copy_(vocab.vectors)

        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=self.dropout, bidirectional=True)
        
    def forward(self, input_seqs, input_lengths, hidden=None):
        if type(input_seqs) == list:
            print('List: ', len(input_seqs))
        else:
            print('np array', input_seqs.shape)
        # Note: we run this all at once (over multiple batches of multiple sequences)
        embedded = self.embedding(input_seqs)

        outputs, hidden = self.gru(embedded, hidden)

        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:] # Sum bidirectional outputs
        return outputs, hidden

## Attention Decoder

### Interpreting the Bahdanau et al. model

The attention model in [Neural Machine Translation by Jointly Learning to Align and Translate](https://arxiv.org/abs/1409.0473) is described as the following series of equations.

Each decoder output is conditioned on the previous outputs and some $\mathbf x$, where $\mathbf x$ consists of the current hidden state (which takes into account previous outputs) and the attention "context", which is calculated below. The function $g$ is a fully-connected layer with a nonlinear activation, which takes as input the values $y_{i-1}$, $s_i$, and $c_i$ concatenated.

$$
p(y_i \mid \{y_1,...,y_{i-1}\},\mathbf{x}) = g(y_{i-1}, s_i, c_i)
$$

The current hidden state $s_i$ is calculated by an RNN $f$ with the last hidden state $s_{i-1}$, last decoder output value $y_{i-1}$, and context vector $c_i$.

In the code, the RNN will be a `nn.GRU` layer, the hidden state $s_i$ will be called `hidden`, the output $y_i$ called `output`, and context $c_i$ called `context`.

$$
s_i = f(s_{i-1}, y_{i-1}, c_i)
$$

The context vector $c_i$ is a weighted sum of all encoder outputs, where each weight $a_{ij}$ is the amount of "attention" paid to the corresponding encoder output $h_j$.

$$
c_i = \sum_{j=1}^{T_x} a_{ij} h_j
$$

... where each weight $a_{ij}$ is a normalized (over all steps) attention "energy" $e_{ij}$ ...

$$
a_{ij} = \dfrac{exp(e_{ij})}{\sum_{k=1}^{T} exp(e_{ik})}
$$

... where each attention energy is calculated with some function $a$ (such as another linear layer) using the last hidden state $s_{i-1}$ and that particular encoder output $h_j$:

$$
e_{ij} = a(s_{i-1}, h_j)
$$

### Implementing an attention module

We implement two layers of attention: `sentence_attn_weights` are weights over sentences and are generated at the <SOS> tag. This is used to weigh word attentions `word_attn_weights`, which are generated at each decoding step.

In [ ]:
class Attn(nn.Module):
    def __init__(self, method, input_size, output_size):
        super(Attn, self).__init__()
        
        self.method = method
        self.input_size = input_size
        self.output_size = output_size

        self.attn = nn.Linear(self.input_size, output_size)
        self.v = nn.Parameter(torch.FloatTensor(1, output_size))

    def forward(self, hidden, encoder_outputs):
        max_len = encoder_outputs.size(0)
        this_batch_size = encoder_outputs.size(1)

        # Create variable to store attention energies
        attn_energies = Variable(torch.zeros(this_batch_size, max_len)) # B x S
        
        if USE_CUDA:
            attn_energies = attn_energies.cuda()

        # For each batch of encoder outputs
        for b in range(this_batch_size):
            # Calculate energy for each encoder output
            for i in range(max_len):
                attn_energies[b, i] = self.score(hidden[b].unsqueeze(0), encoder_outputs[i, b].unsqueeze(0))

        # Normalize energies to weights in range 0 to 1, resize to 1 x B x S
        return F.softmax(attn_energies).unsqueeze(1)
    
    def score(self, hidden, encoder_output):
        
        if self.method == 'dot':
            energy = torch.dot(hidden.view(-1), energy.view(-1))
            return energy
        
        elif self.method == 'general':
            energy = self.attn(encoder_output)
            energy = torch.dot(hidden.view(-1), energy.view(-1))
            
            return energy
        
        elif self.method == 'concat':
#             print ('testing for tuple4', torch.cat((hidden, encoder_output), 1))
            try:
                energy = self.attn(torch.cat((hidden, encoder_output), 1))
            except:
                print ('testing for tuple: hidden:', type(hidden), hidden.shape)
                print ('testing for tuple2: encoder_output', type(encoder_output), encoder_output.shape)
            energy = torch.dot(self.v.contiguous().view(-1), energy.contiguous().view(-1))
            return energy

### Implementing the Bahdanau et al. model

In summary our decoder should consist of four main parts - an embedding layer turning an input word into a vector; a layer to calculate the attention energy per encoder output; a RNN layer; and an output layer.

The decoder's inputs are the last RNN hidden state $s_{i-1}$, last output $y_{i-1}$, and all encoder outputs $h_*$.

* embedding layer with inputs $y_{i-1}$
    * `embedded = embedding(last_rnn_output)`
* attention layer $a$ with inputs $(s_{i-1}, h_j)$ and outputs $e_{ij}$, normalized to create $a_{ij}$
    * `attn_energies[j] = attn_layer(last_hidden, encoder_outputs[j])`
    * `attn_weights = normalize(attn_energies)`
* context vector $c_i$ as an attention-weighted average of encoder outputs
    * `context = sum(attn_weights * encoder_outputs)`
* RNN layer(s) $f$ with inputs $(s_{i-1}, y_{i-1}, c_i)$ and internal hidden state, outputting $s_i$
    * `rnn_input = concat(embedded, context)`
    * `rnn_output, rnn_hidden = rnn(rnn_input, last_hidden)`
* an output layer $g$ with inputs $(y_{i-1}, s_i, c_i)$, outputting $y_i$
    * `output = out(embedded, rnn_output, context)`

In [ ]:
a = torch.zeros(3,1)
torch.equal(a, torch.zeros(a.shape))

In [ ]:
class BahdanauAttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1):
        super(BahdanauAttnDecoderRNN, self).__init__()
        
        # Define parameters
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        # Define layers
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.embedding.weight.data.copy_(vocab.vectors)

        self.dropout = nn.Dropout(dropout_p)
        self.word_attn = Attn('concat', hidden_size * 2, hidden_size)
        self.sentence_attn = Attn('concat', sentence_size + hidden_size, hidden_size) # set at start of notebook
        self.gru = nn.GRU(hidden_size * 2, hidden_size, n_layers, dropout=dropout_p)
        self.out = nn.Linear(hidden_size * 2, output_size)
    
    def forward(self, input_seq, last_hidden, encoder_outputs, sentence_vectors, sentence_idx, sentence_attn_weights):

        # Get the embedding of the current input word (last output word)
        embedded = self.embedding(input_seq)
        embedded = self.dropout(embedded)
        embedded = embedded.unsqueeze(0)
        print(embedded.shape)

        # Calculate attention weights and apply to encoder outputs
        print(encoder_outputs.size())
        print(last_hidden.size())

        # Check if start of sentence
        if torch.sum(input_seq.data) == 0:
            print("Start of new sentence, generating sentence attn weights")
            sentence_attn_weights = self.sentence_attn(last_hidden[-1], sentence_vectors).transpose(0,1)
        print("Sentence attention weight shape: ", sentence_attn_weights.shape)
        
        # wtf are dimensions? (changed so that batch is the middle dimension)
        word_attn_weights = self.word_attn(last_hidden[-1], encoder_outputs).transpose(0,1)
        print("Word attention weight shape: ", word_attn_weights.shape)

        indices_var = sentence_idx.transpose(1,0).unsqueeze(0)
        print("Indices_var: ", indices_var.shape, " max: ", torch.max(indices_var))
        print("Sentence_attn_weights: ", sentence_attn_weights.shape)
        stretched_sent_attn_weights = torch.gather(sentence_attn_weights, 2, indices_var)

        attn_weights = F.softmax(word_attn_weights * stretched_sent_attn_weights, dim=2)
        print("Combined attn weights, ", attn_weights.size())
        
        
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1)) # B x 1 x N
        context = context.transpose(0, 1) # 1 x B x N

        # Combine embedded input word and attended context, run through RNN
        rnn_input = torch.cat((embedded, context), 2)
        output, hidden = self.gru(rnn_input, last_hidden)

        # Final output layer
        output = output.squeeze(0) # B x N
        context = context.squeeze(0) # B x N

        #print("Context size: ", context.size())
        #print("RNN out size: ", output.size())
        #print("Output cat Context size: ", torch.cat((output, context), 1).shape)
        #print("Output size: ", self.out(torch.cat((output, context), 1)).shape)
        output = F.log_softmax(self.out(torch.cat((output, context), 1)))
        
        # Return final output, hidden state, and attention weights (for visualization)
        
        detach_hidden(hidden)
        
        return output, hidden, word_attn_weights, sentence_attn_weights
    
    def detach_hidden(self, hidden):
        if type(hidden) == Variable:
                hidden.detach_() # same as creating a new variable.
        else:
            for h in hidden: h.detach_() 

Now we can build a decoder that plugs this Attn module in after the RNN to calculate attention weights, and apply those weights to the encoder outputs to get a context vector.

### Testing the models

To make sure the Encoder and Decoder model are working (and working together) we'll do a quick test with fake word inputs:

In [ ]:
small_batch_size = 9
b_iter = get_iterator_over_batches(train, small_batch_size)
print('have iter')

for batch in b_iter:
    input_batches = batch.story
    input_lengths = len(batch.story)
    sen_vec = batch.sen_vec.transpose(0,1)
    sen_idx = batch.sen_idx.transpose(0,1)
    target_batches = batch.sum
    target_lengths = len(batch.sum)
    print()
    if USE_CUDA:
        input_batches = input_batches.cuda()
        sen_vec = sen_vec.cuda()
        sen_idx = sen_idx.cuda()
        target_batches = target_batches.cuda()
        
    # input_batches, input_lengths, target_batches, target_lengths = random_batch(small_batch_size)

    print('input_batches', input_batches.size()) # (max_len x batch_size)
    print('sen_vec', sen_vec.size()) # (batch_size x max_num_sen x vec_len)
    print('sen_idx', sen_idx.size()) # (batch_size x max_len)
    print('target_batches', target_batches.size()) # (max_len x batch_size)
    
    break

Create models with a small size (a good idea for eyeball inspection):

In [ ]:
small_hidden_size = 100 #needs to be size of glove vector embeddings?
small_n_layers = 2

encoder_test = EncoderRNN(len(vocab), small_hidden_size, small_n_layers)
decoder_test = BahdanauAttnDecoderRNN(small_hidden_size, len(vocab), small_n_layers)

if USE_CUDA:
    encoder_test.cuda()
    decoder_test.cuda()
    
print(encoder_test)
print(decoder_test)

To test the encoder, run the input batch through to get per-batch encoder outputs:

In [ ]:
encoder_outputs, encoder_hidden = encoder_test(input_batches, input_lengths, None)

print('encoder_outputs', encoder_outputs.size()) # max_len x batch_size x hidden_size
print('encoder_hidden', encoder_hidden.size()) # n_layers * 2 x batch_size x hidden_size

Then starting with a SOS token, run word tokens through the decoder to get each next word token. Instead of doing this with the whole sequence, it is done one at a time, to support using it's own predictions to make the next prediction. This will be one time step at a time, but batched per time step. In order to get this to work for short padded sequences, the batch size is going to get smaller each time.

In [ ]:
max_target_length = target_lengths

# Prepare decoder input and outputs
decoder_input = Variable(torch.LongTensor([vocab.stoi['<SOS>']] * small_batch_size))
print('decoder_input', decoder_input.size())
decoder_hidden = encoder_hidden[:decoder_test.n_layers] # Use last (forward) hidden state from encoder
all_decoder_outputs = Variable(torch.zeros(max_target_length, small_batch_size, decoder_test.output_size))
sentence_attn_weights = Variable(torch.zeros(1, small_batch_size, sen_vec.shape[0]))

print ('sen_vec', sen_vec.shape)
print ('sen_idx', sen_idx.shape)
print ('sentence_attn_weights', sentence_attn_weights.shape)

if USE_CUDA:
    all_decoder_outputs = all_decoder_outputs.cuda()
    decoder_input = decoder_input.cuda()

# Run through decoder one time step at a time
for t in range(target_lengths):
    decoder_output, decoder_hidden, word_attn_weights, sentence_attn_weights = decoder_test(
        decoder_input.contiguous(), decoder_hidden.contiguous(), encoder_outputs.contiguous(),
        sen_vec, sen_idx, sentence_attn_weights
    )
    all_decoder_outputs[t] = decoder_output # Store this step's outputs
    decoder_input = target_batches[t] # Next input is current target

    
print(all_decoder_outputs.size())
print(target_batches.size())

# Test masked cross entropy loss
loss = masked_cross_entropy(
    all_decoder_outputs.transpose(0, 1).contiguous(),
    target_batches.transpose(0, 1).contiguous(),
    torch.LongTensor([vocab.stoi['<SOS>']] * target_batches.size(1)),
    use_cuda=USE_CUDA
)
print('loss', loss.data[0])

In [ ]:
a = torch.zeros(1,2,3)
b = torch.LongTensor([[[1,2,0,1],[1,0,1,1]]])
print(a.shape, b.shape)
torch.gather(a, 2, b)

# Training

## Defining a training iteration

To train we first run the input sentence through the encoder word by word, and keep track of every output and the latest hidden state. Next the decoder is given the last hidden state of the decoder as its first hidden state, and the `<SOS>` token as its first input. From there we iterate to predict a next token from the decoder.

### Teacher Forcing and Scheduled Sampling

"Teacher Forcing", or maximum likelihood sampling, means using the real target outputs as each next input when training. The alternative is using the decoder's own guess as the next input. Using teacher forcing may cause the network to converge faster, but [when the trained network is exploited, it may exhibit instability](http://minds.jacobs-university.de/sites/default/files/uploads/papers/ESNTutorialRev.pdf).

You can observe outputs of teacher-forced networks that read with coherent grammar but wander far from the correct translation - you could think of it as having learned how to listen to the teacher's instructions, without learning how to venture out on its own.

The solution to the teacher-forcing "problem" is known as [Scheduled Sampling](https://arxiv.org/abs/1506.03099), which simply alternates between using the target values and predicted values when training. We will randomly choose to use teacher forcing with an if statement while training - sometimes we'll feed use real target as the input (ignoring the decoder's output), sometimes we'll use the decoder's output.

In [ ]:
decoder_input = Variable(torch.LongTensor([SOS_token] * 5))
torch.equal(decoder_input.data, torch.LongTensor([SOS_token] * 5))

In [ ]:
a = Variable(torch.FloatTensor(np.array([[1,2,3],[3,2,6]])))
torch.max(a.data)

In [ ]:
def train(input_batches, input_lengths, target_batches, target_lengths, sentence_batches, sentence_lengths,
          encoder, decoder, encoder_optimizer, decoder_optimizer, criterion):
    
    # Zero gradients of both optimizers
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0 # Added onto for each word
    # Run words through encoder
    encoder_outputs, encoder_hidden = encoder(input_batches, input_lengths, None)
    # Prepare input and output variables
    decoder_input = Variable(torch.LongTensor([SOS_token] * batch_size))
    decoder_hidden = encoder_hidden[:decoder.n_layers] # Use last (forward) hidden state from encoder
    max_target_length = max(target_lengths)
    all_decoder_outputs = Variable(torch.zeros(max_target_length, batch_size, decoder.output_size))
    sentence_attn_weights = Variable(torch.zeros(1, batch_size, torch.max(sentence_lengths))
    print('max_target_length = ' + str(max_target_length))
    
    # Move new Variables to CUDA
    if USE_CUDA:
        decoder_input = decoder_input.cuda()
        all_decoder_outputs = all_decoder_outputs.cuda()

    # Run through decoder one time step at a time
    for t in range(max_target_length):
        decoder_output, decoder_hidden, decoder_attn = decoder(
            decoder_input, decoder_hidden, encoder_outputs, sentence_batches
        )

        all_decoder_outputs[t] = decoder_output
        decoder_input = target_batches[t] # Next input is current target
    # Loss calculation and backpropagation
    loss = masked_cross_entropy(
        all_decoder_outputs.transpose(0, 1).contiguous(), # -> batch x seq
        target_batches.transpose(0, 1).contiguous(), # -> batch x seq
        target_lengths
    )
    loss.backward()
    
    # Clip gradient norms
    ec = torch.nn.utils.clip_grad_norm(encoder.parameters(), clip)
    dc = torch.nn.utils.clip_grad_norm(decoder.parameters(), clip)

    # Update parameters with optimizers
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.data[0], ec, dc

Finally helper functions to print time elapsed and estimated time remaining, given the current time and progress.

In [ ]:
def as_minutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def time_since(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (as_minutes(s), as_minutes(rs))

## Running training

With everything in place we can actually initialize a network and start training.

To start, we initialize models, optimizers, and a loss function (criterion).

In [ ]:
hidden_size = 500
n_layers = 2
dropout = 0.1
batch_size = 100
#batch_size = 50
dropout= .1
# Initialize models
encoder = EncoderRNN(voc.n_words, hidden_size, n_layers, dropout=dropout)
decoder = BahdanauAttnDecoderRNN(hidden_size, voc.n_words, n_layers)

# Move models to GPU
if USE_CUDA:
    encoder.cuda()
    decoder.cuda()

# if torch.cuda.device_count() > 1:
#     encoder = nn.DataParallel(encoder)
#     decoder = nn.DataParallel(decoder)





Setting parameters for training

In [ ]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 0.5
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_epochs = 50000
epoch = 0
plot_every = 20
print_every = 2
evaluate_every = 100


# Initialize optimizers and criterion
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
criterion = nn.CrossEntropyLoss()

print out the progress while training

In [ ]:

# Keep track of time elapsed and running averages
start = time.time()
plot_losses = []
print_loss_total = 2 # Reset every print_every
plot_loss_total = 2 # Reset every plot_every

# Evaluating the network

Evaluation is mostly the same as training, but there are no targets. Instead we always feed the decoder's predictions back to itself. Every time it predicts a word, we add it to the output string. If it predicts the EOS token we stop there. We also store the decoder's attention outputs for each step to display later.

In [ ]:
def evaluate(input_seq):
    input_lengths = [len(input_seq)]
    input_seqs = [indexes_from_sentence(input_lang, input_seq)]
    input_batches = Variable(torch.LongTensor(input_seqs), volatile=True).transpose(0, 1)
    
    if USE_CUDA:
        input_batches = input_batches.cuda()
        
    # Set to not-training mode to disable dropout
    encoder.train(False)
    decoder.train(False)
    
    # Run through encoder
    encoder_outputs, encoder_hidden = encoder(input_batches, input_lengths, None)

    # Create starting vectors for decoder
    decoder_input = Variable(torch.LongTensor([SOS_token]), volatile=True) # SOS
    decoder_hidden = encoder_hidden[:decoder.n_layers] # Use last (forward) hidden state from encoder
    
    if USE_CUDA:
        decoder_input = decoder_input.cuda()

    # Store output words and attention states
    decoded_words = []
    decoder_attentions = torch.zeros(max_length + 1, max_length + 1)
    
    # Run through decoder
    for di in range(max_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs
        )
        decoder_attentions[di,:decoder_attention.size(2)] += decoder_attention.squeeze(0).squeeze(0).cpu().data

        # Choose top word from output
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(output_lang.index2word[ni])
            
        # Next input is chosen word
        decoder_input = Variable(torch.LongTensor([ni]))
        if USE_CUDA: decoder_input = decoder_input.cuda()

    # Set back to training mode
    encoder.train(True)
    decoder.train(True)
    
    return decoded_words, decoder_attentions[:di+1, :len(encoder_outputs)]

We can evaluate random sentences from the training set and print out the input, target, and output to make some subjective quality judgements:

In [ ]:
def evaluate_randomly():
    pair = random.choice(pairs)
    evaluate_and_show_attention(pair['story'], pair['summary'])

# Visualizing attention

A useful property of the attention mechanism is its highly interpretable outputs. Because it is used to weight specific encoder outputs of the input sequence, we can imagine looking where the network is focused most at each time step.

You could simply run `plt.matshow(attentions)` to see attention output displayed as a matrix, with the columns being input steps and rows being output steps:

In [ ]:
# import io
# import torchvision
# from PIL import Image
# import visdom
# vis = visdom.Visdom()

# def show_plot_visdom():
#     buf = io.BytesIO()
#     plt.savefig(buf)
#     buf.seek(0)
#     attn_win = 'attention (%s)' % hostname
#     vis.image(torchvision.transforms.ToTensor()(Image.open(buf)), win=attn_win, opts={'title': attn_win})

For a better viewing experience we will do the extra work of adding axes and labels:

In [ ]:
def show_attention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') + ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    #show_plot_visdom()
    plt.show()
    plt.close()

In [ ]:

# def evaluate_and_show_attention(input_sentence, target_sentence=None):
#     output_words, attentions = evaluate(input_sentence)
#     output_sentence = ' '.join(output_words)
#     print('>', input_sentence)
#     if target_sentence is not None:
#         print('=', target_sentence)
#     print('<', output_sentence)
    
#     show_attention(input_sentence, output_words, attentions)
    
#     # Show input, target, output text in visdom
#     win = 'evaluted (%s)' % hostname
#     text = '<p>&gt; %s</p><p>= %s</p><p>&lt; %s</p>' % (input_sentence, target_sentence, output_sentence)
#     vis.text(text, win=win, opts={'title': win})

# Putting it all together

**TODO** Run `train_epochs` for `n_epochs`

To actually train, we call the train function many times, printing a summary as we go.

*Note:* If you're running this notebook you can **train, interrupt, evaluate, and come back to continue training**. Simply run the notebook starting from the following cell (running from the previous cell will reset the models).

In [ ]:
# Begin!
ecs = []
dcs = []
eca = 0
dca = 0

while epoch < n_epochs:
    epoch += 1
    
    # Get training data for this cycle
    input_batches, input_lengths, target_batches, target_lengths, sentence_batches, sentence_lengths = random_batch(batch_size)

    # Run the train function
    loss, ec, dc = train(
        input_batches, input_lengths, target_batches, target_lengths, sentence_batches, sentence_lengths,
        encoder, decoder,
        encoder_optimizer, decoder_optimizer, criterion
    )

    # Keep track of loss
    print_loss_total += loss
    plot_loss_total += loss
    eca += ec
    dca += dc
    
    print_loss_avg = print_loss_total / print_every
    print_loss_total = 0
    print_summary = '%s (%d %d%%) %.4f' % (time_since(start, epoch / n_epochs), epoch, epoch / n_epochs * 100, print_loss_avg)
    print(print_summary)

    ##job.record(epoch, loss)

#     if epoch % 1 == 0:
#         print_loss_avg = print_loss_total / print_every
#         print_loss_total = 0
#         print_summary = '%s (%d %d%%) %.4f' % (time_since(start, epoch / n_epochs), epoch, epoch / n_epochs * 100, print_loss_avg)
#         print(print_summary)
        
    if epoch % evaluate_every == 0:
        evaluate_randomly()

    if epoch % plot_every == 0:
        plot_loss_avg = plot_loss_total / plot_every
        plot_losses.append(plot_loss_avg)
        plot_loss_total = 0
        
        # TODO: Running average helper
        ecs.append(eca / plot_every)
        dcs.append(dca / plot_every)
        
        '''
        ecs_win = 'encoder grad (%s)' % hostname
        dcs_win = 'decoder grad (%s)' % hostname
        vis.line(np.array(ecs), win=ecs_win, opts={'title': ecs_win})
        vis.line(np.array(dcs), win=dcs_win, opts={'title': dcs_win})
        '''

        eca = 0
        dca = 0

## Plotting training loss

Plotting is done with matplotlib, using the array `plot_losses` that was created while training.

In [ ]:
def show_plot(points):
    plt.figure()
    fig, ax = plt.subplots()
    loc = ticker.MultipleLocator(base=0.2) # put ticks at regular intervals
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

show_plot(plot_losses)

In [ ]:
output_words, attentions = evaluate("premier romano prodi battled tuesday for any votes freed up from a split in a far left party but said he will resign if he loses a confidence vote expected later this week .")
plt.matshow(attentions.numpy())
show_plot_visdom()

In [ ]:
evaluate_and_show_attention("a south korean lawmaker said friday communist north korea could be producing plutonium and could have more secret underground nuclear facilities than already feared .")

In [ ]:
evaluate_and_show_attention("egyptian president hosni mubarak met here sunday with syrian president hafez assad to try to defuse growing tension between syria and turkey .")

In [ ]:
evaluate_and_show_attention("police and soldiers on friday blocked off the street in front of a house where members of a terrorist gang are believed to have assembled the bomb that blew up the u .s . embassy killing people .")

In [ ]:
evaluate_and_show_attention("premier romano prodi battled tuesday for any votes freed up from a split in a far left party but said he will resign if he loses a confidence vote expected later this week .")

# To do

* Try with a different dataset
    * cnn/dailymail
    * gigawords
    * standford
    * Human &rarr; Machine (e.g. IOT commands)
    * Chat &rarr; Response
    * Question &rarr; Answer
* Replace the embedding pre-trained word embeddings such as word2vec or GloVe
* Try with more layers, more hidden units, and more sentences. Compare the training time and results.
* Try different RNN layers like lstm.
* Add batch operation for GPU training
* Add beam search on decoder side when dealing with long documents.
* Control the Different output size
* Dig out other tricks